<a href="https://colab.research.google.com/github/AKASH4671/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/blob/main/03_sentiment_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**mount drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**libraries**

In [ ]:
!pip install transformers
!pip install torch

import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

**load data**

In [ ]:
# Load the cleaned data from previous step
file_path = "/content/drive/MyDrive/Colab Notebooks/Finance Projects/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/data/apple_news_cleaned.csv"
df = pd.read_csv(file_path)

print("Loaded data:", df.shape)
df.head()

Loaded data: (2934, 5)


,date,headline,cleaned_text,word_count,weekday
0,2025-06-24 19:59:58,Node v22.17.0 (LTS),node lts nodejs free opensource crossplatform ...,38,Tuesday
1,2025-06-24 19:57:16,"Marcelo vê Moçambique a ""olhar para o futuro""",marcelo see mozambique look futurepresident re...,36,Tuesday
2,2025-06-24 19:56:34,Obediências maçónicas emitem declaração em def...,masonic obedience issue statement defense peac...,38,Tuesday
3,2025-06-24 19:54:57,8点1氪｜顺丰等多家快递公司拒收罗马仕充电宝；字节通报大模型团队负责人出轨HRBP处理结果；...,express express delivery company refused accep...,124,Tuesday
4,2025-06-24 19:54:56,Almada. Detido suspeito de tentativa de homicí...,almadadetained suspected attempted murder braz...,36,Tuesday


In [ ]:
# NaN or empty cleaned_text
print("Missing cleaned_text:", df['cleaned_text'].isna().sum())
print("Empty cleaned_text rows:", df['cleaned_text'].str.strip().eq('').sum())

# Duplicates
print("Duplicate articles:", df.duplicated(subset=['headline', 'cleaned_text']).sum())

Missing cleaned_text: 0
Empty cleaned_text rows: 0
Duplicate articles: 0


**load FinBERT Model from Hugging Face**

In [ ]:
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create sentiment pipeline
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Device set to use cpu


**Apply Sentiment Scoring to news**

In [ ]:
# Apply FinBERT on cleaned_text
def get_sentiment(text):
    if isinstance(text, str) and text.strip() != "":
        return nlp(text[:512])[0]  # truncate to 512 tokens
    else:
        return {'label': 'neutral', 'score': 0.0}

# Run sentiment analysis
sentiment_output = df['cleaned_text'].apply(get_sentiment)

# Separate into columns
df['sentiment'] = sentiment_output.apply(lambda x: x['label'])
df['sentiment_score'] = sentiment_output.apply(lambda x: x['score'])

print("Sentiment scoring complete.")
df[['headline', 'sentiment', 'sentiment_score']].head()

Sentiment scoring complete.


,headline,sentiment,sentiment_score
0,Node v22.17.0 (LTS),Neutral,0.999738
1,"Marcelo vê Moçambique a ""olhar para o futuro""",Neutral,0.999268
2,Obediências maçónicas emitem declaração em def...,Neutral,0.999991
3,8点1氪｜顺丰等多家快递公司拒收罗马仕充电宝；字节通报大模型团队负责人出轨HRBP处理结果；...,Neutral,0.999938
4,Almada. Detido suspeito de tentativa de homicí...,Neutral,0.999965


In [ ]:
print("Sentiment Distribution:")
print(df['sentiment'].value_counts())

Sentiment Distribution:
sentiment
Neutral     2435
Positive     393
Negative     106
Name: count, dtype: int64


**save**

In [ ]:
# Save final sentiment-scored news data
output_path = "/content/drive/MyDrive/Colab Notebooks/Finance Projects/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/data/apple_sentiment_dataset.csv"
df.to_csv(output_path, index=False)

print(f"Saved final sentiment dataset to: {output_path}")

Saved final sentiment dataset to: /content/drive/MyDrive/Colab Notebooks/Finance Projects/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/data/apple_sentiment_dataset.csv
